In [1]:
import sys, os, time
import numpy as np
import scipy.io as sio

In [2]:
# Accuracy metric
def acc(r):
    acc = round(r[len(r)-1].sum() / len(r), 4)
    acc_std = round(r[len(r)-1].std(), 4)
    return acc, acc_std
# Backward Transfer metric
def bwt(r):
    t = len(r)
    tmp = []
    for i in range(t-1):
        tmp.append((r[t-1][i] - r[i][i]))
    tmp_arr = np.array(tmp)
    bwt = round(tmp_arr.sum()/(t-1), 4)
    bwt_std = round(tmp_arr.std(), 4)
    return bwt, bwt_std

In [3]:
exp = 'minigrid-wallgap-doorkey-redbluedoor-crossing'
steps = '500000'
approaches = ['blip','ewc','fine-tuning','blip_ewc_1', 'blip_ewc_2', 'blip_spp_mask']
stages = ['train', 'eval']
seeds = [123456, 789012, 345678]
F_prior = 5e-18
ewc_lambda = 5000.0

blip_ewc_lambda_1 = 1000.0
blip_ewc_lambda_2 = 2500.0
fisher_term = 'ft'

spp_lambda = 0.5
initial_prune_percent = 30.0
scheduler = False
prune_higher = False

# Experiment details
experiments = [
    ('2023-11-13', exp, [
    (0, 'MiniGrid-WallGapS6-v0'),
    (1, 'MiniGrid-DoorKey-6x6-v0'),
    (2, 'MiniGrid-RedBlueDoors-6x6-v0'), 
    (3, 'MiniGrid-SimpleCrossingS9N1-v0')  
    ])    
    ]

for item in experiments:

    date = item[0]
    experiment = item[1]
    tasks_sequence = item[2]
    t = len(tasks_sequence)

    print('Experiment:',experiment, '\n')
    for approach in approaches:
        r_all = []
        for seed in seeds:
            r = np.zeros([len(tasks_sequence), len(tasks_sequence)])
            if approach == 'fine-tuning' or approach == 'ft-fix':
                log_name = '{}_{}_{}_{}'.format(date, experiment, approach, seed)
            elif approach == 'ewc' in approach:
                log_name = '{}_{}_{}_{}_{}_lamb_{}'.format(date, experiment, approach, seed, steps, ewc_lambda)
            elif approach == 'blip':
                log_name = '{}_{}_{}_{}_{}_F_prior_{}'.format(date, experiment, approach, seed, steps, F_prior)
            elif approach == 'blip_ewc_1':
                log_name = '{}_{}_{}_{}_{}_F_prior_{}_lamb_{}_F_term_{}'.format(date, experiment, 'blip_ewc', seed, steps, F_prior, blip_ewc_lambda_1, fisher_term)
            elif approach == 'blip_ewc_2':
                log_name = '{}_{}_{}_{}_{}_F_prior_{}_lamb_{}_F_term_{}'.format(date, experiment, 'blip_ewc', seed, steps, F_prior, blip_ewc_lambda_2, fisher_term)
            elif approach == 'blip_spp_mask':
                log_name = '{}_{}_{}_{}_{}_F_prior_{}_spp_lamb_{}_prune_{}_scheduler_{}_prune_higher_{}'.format(date, experiment, approach, seed, steps, F_prior, spp_lambda, initial_prune_percent, scheduler, prune_higher)

            mat_file = os.path.join('./result_data/', log_name + '_result.mat')
            mat_contents = sio.loadmat(mat_file)
            te_reward_arr = mat_contents['te_reward_arr'][0][0][0][0][0][0][0]
            num_samples = len(te_reward_arr[0][0])
            len_task = int(num_samples / t)
            for i in range(t):
                index = (i+1)*len_task - 1
                for j in range(t):        
                    r[i][j] = te_reward_arr[j][0][index]
            r_all.append(r)

        r_median = np.median(r_all, axis=0)
        print("Approach:", approach)
        print("ACC: {}".format(acc(r_median)))
        print("BWT: {}\n".format(bwt(r_median)))

Experiment: minigrid-wallgap-doorkey-redbluedoor-crossing 



KeyboardInterrupt: 